In [1]:
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load data from CSV file (replace with your actual CSV file path)
data = pd.read_csv('/teamspace/studios/this_studio/data/csv-clean/synthetic000-cleaned.csv')

# Load classifications from JSON file (replace with your actual JSON file path)
classifications_file_path = '/teamspace/studios/this_studio/csvandmetadata export /json/classifications_0000.json'
with open(classifications_file_path, 'r') as f:
    classifications = json.load(f)

# Define a function to apply operations based on classifications
def apply_operations_based_on_classification(data, classifications):
    # Convert classifications to lowercase keys for easier matching
    classifications_lower = {k.lower(): v for k, v in classifications.items()}
    
    # Identify numeric columns
    numeric_columns = data.select_dtypes(include='number')
    
    # Calculate min-max range for numeric columns
    min_max_range = numeric_columns.max() - numeric_columns.min()
    
    # Initialize scalers
    scaler_standard = StandardScaler()
    scaler_minmax = MinMaxScaler()



    # Process each column in the DataFrame
    for col in data.columns:
        col_lower = col.lower()  # Convert column name to lowercase for matching
        
        if col_lower in classifications_lower:
            classification = classifications_lower[col_lower]
            
            if classification == 'numeric data':
                if min_max_range.mean() < 10:
                    # Use MinMaxScaler for normalization if range is small
                    data[col] = scaler_minmax.fit_transform(data[[col]])
                elif numeric_columns[col].abs().skew() > 0.5:
                    # Use StandardScaler for standardization if data is skewed
                    data[col] = scaler_standard.fit_transform(data[[col]])
                else:
                    # Default to StandardScaler if no specific conditions are met
                    data[col] = scaler_standard.fit_transform(data[[col]])
            
            # Add conditions for other classifications if needed
            elif classification == 'categorical data':
                # Handle categorical data (if needed)
                pass
            elif classification == 'temporal data':
                # Handle temporal data (if needed)
                pass

    return data

# Apply operations based on classifications
data_processed = apply_operations_based_on_classification(data, classifications)

# Example: Print first few rows of processed data
print(data_processed.head())

# Example: Save the processed data back to CSV
data_processed.to_csv('processed_combined_data.csv', index=False)


   case_id  Hospital_code Hospital_type_code  City_Code_Hospital  \
0        1              8                  c             3.00000   
1        2              2                  c             5.00000   
2        3             10                  e             1.00000   
3        4             26                  b             4.69814   
4        5             26                  b             2.00000   

  Hospital_region_code  Available Extra Rooms in Hospital    Department  \
0                    Z                                3.0  radiotherapy   
1                    Z                                2.0  radiotherapy   
2                    X                                2.0    anesthesia   
3                    Y                                2.0  radiotherapy   
4                    Y                                2.0  radiotherapy   

  Ward_Type Ward_Facility_Code Bed Grade  patientid  City_Code_Patient  \
0         R                  F       2.0      31397               

categerial what can we do immidiate is 
1. is sring numeric values are cast as numeric values like age and stay
2. if nunique are less tha n5 or equal to five one hot
3. lable encoding whole dataset 

Non-Ordinal Data for Non-Tree-Based Models:

For algorithms like linear regression, logistic regression, k-nearest neighbors (KNN), support vector machines (SVM), and neural networks, label encoding can cause problems if the data is not ordinal. These algorithms can interpret the encoded values as having an inherent order or relationship, which might not be accurate.

so below code for non ordinal opration 

in earlier combile floate and inter in numeric daata 

In [1]:
# script for non ordinal data   

import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder  

# Load data from CSV file (replace with your actual CSV file path)
data = pd.read_csv('/teamspace/studios/this_studio/data/csv-clean/synthetic000-cleaned.csv')

# Load classifications from JSON file (replace with your actual JSON file path)
classifications_file_path = '/teamspace/studios/this_studio/csvandmetadata export /json/classifications_0000.json'
with open(classifications_file_path, 'r') as f:
    classifications = json.load(f)

# Define a function to apply operations based on classifications
def apply_operations_based_on_classification(data, classifications):
    # Convert classifications to lowercase keys for easier matching
    classifications_lower = {k.lower(): v for k, v in classifications.items()}
    
    # Identify numeric columns
    numeric_columns = data.select_dtypes(include='number')
    
    # Calculate min-max range for numeric columns
    min_max_range = numeric_columns.max() - numeric_columns.min()
    
    # Initialize scalers
    scaler_standard = StandardScaler()
    scaler_minmax = MinMaxScaler()
    le = LabelEncoder()


    # Process each column in the DataFrame
    for col in data.columns:
        col_lower = col.lower()  # Convert column name to lowercase for matching
        
        if col_lower in classifications_lower:
            classification = classifications_lower[col_lower]
            
            if classification == 'numeric data':
                if min_max_range.mean() < 10:
                    # Use MinMaxScaler for normalization if range is small
                    data[col] = scaler_minmax.fit_transform(data[[col]])
                elif numeric_columns[col].abs().skew() > 0.5:
                    # Use StandardScaler for standardization if data is skewed
                    data[col] = scaler_standard.fit_transform(data[[col]])
                else:
                    # Default to StandardScaler if no specific conditions are met
                    data[col] = scaler_standard.fit_transform(data[[col]])
            
            # Add conditions for other classifications if needed
            elif classification == 'categorical data':
                if  data[col].nunique() < 10:
                    data = pd.get_dummies(data, columns=[col])
                elif data[col].nunique() > 10:
                    data[col] = data[col].astype('category').cat.codes
                
            
            elif classification == 'email data':
                if data[col].str.contains('@').any():  # Ensure it's email data
                    data[col] = data[col].apply(hash_email)
                
            elif classification == 'temporal data': #TIME OF DATES
                data[col] = pd.to_datetime(data[col])
                data[col + '_year'] = data[col].dt.year
                data[col + '_month'] = data[col].dt.month
                data[col + '_day'] = data[col].dt.day
                data[col + '_hour'] = data[col].dt.hour
                data[col + '_minute'] = data[col].dt.minute
                data[col + '_second'] = data[col].dt.second
                data = data.drop(columns=[col])
                
            elif classfication ==  'ID Data':
                if data[col].nunique() == data.shape[0]:
                   data = data.drop(columns=[col])
                elif pd.api.types.is_numeric_dtype(data[col]):
                   data[[col]] = scaler_minmax.fit_transform(data[[col]])
                elif data[col].dtype == 'object':
                   data[col] = data[col].astype('category').cat.codes

            elif classification == 'location data':
                data[col] = data[col].astype('category').cat.codes

            elif classification == 'phone number data':
                data[col] = data[col].astype('category').cat.codes

            elif classsification == 'binary data':
                if data[col].nunique() == 2:
                   data[col] = le.fit_transform(data[col])

            
                

    return data

# Apply operations based on classifications
data_processed = apply_operations_based_on_classification(data, classifications)

# Example: Print first few rows of processed data
print(data_processed.head())

# Example: Save the processed data back to CSV
data_processed.to_csv('processed_combined_data.csv', index=False)


   case_id  Hospital_code Hospital_type_code  City_Code_Hospital  \
0        1              8                  c             3.00000   
1        2              2                  c             5.00000   
2        3             10                  e             1.00000   
3        4             26                  b             4.69814   
4        5             26                  b             2.00000   

  Hospital_region_code  Available Extra Rooms in Hospital    Department  \
0                    Z                                3.0  radiotherapy   
1                    Z                                2.0  radiotherapy   
2                    X                                2.0    anesthesia   
3                    Y                                2.0  radiotherapy   
4                    Y                                2.0  radiotherapy   

  Ward_Type Ward_Facility_Code Bed Grade  patientid  City_Code_Patient  \
0         R                  F       2.0      31397               

For algorithms like decision trees, random forests, and gradient boosting, label encoding can be effective even if the data is not ordinal because these models are not sensitive to the numerical ordering of labels. These models can handle categorical features encoded as integers.

we will use 

In [6]:
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Load data from CSV file (replace with your actual CSV file path)
data = pd.read_csv('/teamspace/studios/this_studio/data/csv-clean/synthetic565-cleaned.csv')

# Load classifications from JSON file (replace with your actual JSON file path)
classifications_file_path = '/teamspace/studios/this_studio/csvandmetadata export /json/claasification_555.json'
with open(classifications_file_path, 'r') as f:
    classifications = json.load(f)

# Define a function to apply operations based on classifications
def apply_operations_based_on_classification(data, classifications):
    # Convert classifications to lowercase keys for easier matching
    classifications_lower = {k.lower(): v for k, v in classifications.items()}
    
    # Identify numeric columns
    numeric_columns = data.select_dtypes(include='number')
    
    # Calculate min-max range for numeric columns
    min_max_range = numeric_columns.max() - numeric_columns.min()
    
    # Initialize scalers
    scaler_standard = StandardScaler()
    scaler_minmax = MinMaxScaler()

    # Compute skewness for numeric columns
    skewness = numeric_columns.skew()

    # Process each column in the DataFrame
    for col in data.columns:
        col_lower = col.lower()  # Convert column name to lowercase for matching
        
        if col_lower in classifications_lower:
            classification = classifications_lower[col_lower]
            
            if classification == 'numeric data':
                if min_max_range[col] < 10:
                    # Use MinMaxScaler for normalization if range is small
                    data[col] = scaler_minmax.fit_transform(data[[col]])
                elif skewness[col] > 0.5:
                    # Use StandardScaler for standardization if data is skewed
                    data[col] = scaler_standard.fit_transform(data[[col]])
                else:
                    # Default to StandardScaler if no specific conditions are met
                    data[col] = scaler_standard.fit_transform(data[[col]])
            
            # Add conditions for other classifications if needed
            elif classification == 'categorical data':
                if data[col].nunique() < 10:
                    data = pd.get_dummies(data, columns=[col])
                else:
                    le = LabelEncoder()
                    data[col] = le.fit_transform(data[col])
            
            # Handle additional classifications if needed
            # elif classification == 'temporal data':
            #     # Handle temporal data (if needed)
            #     pass

    return data

# Apply operations based on classifications
data_processed = apply_operations_based_on_classification(data, classifications)

# Example: Print first few rows of processed data
print(data_processed.head())

# Example: Save the processed data back to CSV
data_processed.to_csv('processed_combined_data_final_2.csv', index=False)


   case_id  Hospital_code Hospital_type_code  City_Code_Hospital  \
0        1              8                  c             3.00000   
1        2              2                  c             5.00000   
2        3             10                  e             1.00000   
3        4             26                  b             4.69814   
4        5             26                  b             2.00000   

  Hospital_region_code  Available Extra Rooms in Hospital    Department  \
0                    Z                                3.0  radiotherapy   
1                    Z                                2.0  radiotherapy   
2                    X                                2.0    anesthesia   
3                    Y                                2.0  radiotherapy   
4                    Y                                2.0  radiotherapy   

  Ward_Type Ward_Facility_Code Bed Grade  patientid  City_Code_Patient  \
0         R                  F       2.0      31397               

In [2]:
#fix categorical data

import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Load data from CSV file
data = pd.read_csv('/teamspace/studios/this_studio/data/csv-clean/synthetic565-cleaned.csv')

# Load classifications from JSON file
classifications_file_path = '/teamspace/studios/this_studio/csvandmetadata export /json/claasification_555.json'
with open(classifications_file_path, 'r') as f:
    classifications = json.load(f)

# Define a function to apply operations based on classifications
def apply_operations_based_on_classification(data, classifications):
    # Initialize scalers
    scaler_standard = StandardScaler()
    scaler_minmax = MinMaxScaler()

    # Process each column in the DataFrame
    for col in data.columns:
        col_lower = col.lower()  # Convert column name to lowercase for matching

        # Find the classification for the column
        classification = None
        for key, value in classifications.items():
            if col in value:
                classification = key
                break

        if classification is None:
            continue  # Skip if no classification is found

        if classification == 'Numeric Data':
            # Calculate min-max range and skewness for the column
            min_max_range = data[col].max() - data[col].min()
            skewness = data[col].skew()

            if min_max_range < 10:
                # Use MinMaxScaler for normalization if range is small
                data[col] = scaler_minmax.fit_transform(data[[col]])
            elif skewness > 0.5:
                # Use StandardScaler for standardization if data is skewed
                data[col] = scaler_standard.fit_transform(data[[col]])
            else:
                # Default to StandardScaler if no specific conditions are met
                data[col] = scaler_standard.fit_transform(data[[col]])

        elif classification == 'Categorical Data':
            if data[col].nunique() < 10:
                # Use one-hot encoding if unique values are less than 10
                data = pd.get_dummies(data, columns=[col], drop_first=True)
            else:
                # Use label encoding if unique values are 10 or more
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])

        # Add conditions for other classifications if needed
        # elif classification == 'Temporal Data':
        #     # Handle temporal data (if needed)
        #     pass

    return data

# Apply operations based on classifications
data_processed = apply_operations_based_on_classification(data, classifications)

# Example: Print first few rows of processed data
print(data_processed.head())

# Example: Save the processed data back to CSV
data_processed.to_csv('/teamspace/studios/this_studio/data/csv-clean/processed_combined_data_final_2.csv', index=False)


    case_id  Hospital_code  City_Code_Hospital  \
0 -1.735116      -1.244013           -0.524763   
1 -1.734423      -1.932504            0.093487   
2 -1.733729      -1.014516           -1.143013   
3 -1.733036       0.821461            0.000174   
4 -1.732343       0.821461           -0.833888   

   Available Extra Rooms in Hospital  patientid  City_Code_Patient  \
0                           0.222222  -0.965397           0.029048   
1                           0.111111  -0.965397           0.029048   
2                           0.111111  -0.965397           0.029048   
3                           0.111111  -0.965397           0.029048   
4                           0.111111  -0.965397           0.029048   

   Visitors with Patient       Age  Admission_Deposit  Stay  ...  \
0              -0.736972  0.246822           0.016381     4  ...   
1              -0.736972  0.246822           1.034451     3  ...   
2              -0.736972  0.246822          -0.145651     2  ...   
3     

In [4]:
# really final script for all data types
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import hashlib

# Load data from CSV file
data = pd.read_csv('/teamspace/studios/this_studio/data/csv-clean/synthetic565-cleaned.csv')

# Load classifications from JSON file
classifications_file_path = '/teamspace/studios/this_studio/csvandmetadata export /json/claasification_555.json'
with open(classifications_file_path, 'r') as f:
    classifications = json.load(f)

# Define a function to apply operations based on classifications
def apply_operations_based_on_classification(data, classifications):
    # Initialize scalers
    scaler_standard = StandardScaler()
    scaler_minmax = MinMaxScaler()

    # Process each column in the DataFrame
    for col in data.columns:
        col_lower = col.lower()  # Convert column name to lowercase for matching

        # Find the classification for the column
        classification = None
        for key, value in classifications.items():
            if col in value:
                classification = key
                break

        if classification is None:
            continue  # Skip if no classification is found

        if classification == 'Numeric Data':
            # Calculate min-max range and skewness for the column
            min_max_range = data[col].max() - data[col].min()
            skewness = data[col].skew()

            if min_max_range < 10:
                # Use MinMaxScaler for normalization if range is small
                data[col] = scaler_minmax.fit_transform(data[[col]])
            elif skewness > 0.5:
                # Use StandardScaler for standardization if data is skewed
                data[col] = scaler_standard.fit_transform(data[[col]])
            else:
                # Default to StandardScaler if no specific conditions are met
                data[col] = scaler_standard.fit_transform(data[[col]])

        elif classification == 'Categorical Data':
            if data[col].nunique() < 10:
                # Use one-hot encoding if unique values are less than 10
                data = pd.get_dummies(data, columns=[col], drop_first=True)
            else:
                # Use label encoding if unique values are 10 or more
                le = LabelEncoder()
                data[col] = le.fit_transform(data[col])


        
            
        #elif classification == 'Email Data':
            #email opration to be happen here
            #data[col] = data[col].apply(hash_email)
                
        # Convert boolean columns to 0 and 1
        bool_columns = data.select_dtypes(include='bool').columns
        data[bool_columns] = data[bool_columns].astype(int)

        # Add conditions for other classifications if needed
        # elif classification == 'Temporal Data':
        #     # Handle temporal data (if needed)
        #     pass

    return data

# Apply operations based on classifications
data_processed = apply_operations_based_on_classification(data, classifications)

# Example: Print first few rows of processed data
print(data_processed.head())

# Example: Save the processed data back to CSV
data_processed.to_csv('/teamspace/studios/this_studio/data/csv-clean/processed_combined_data_final_3.csv', index=False)


    case_id  Hospital_code  City_Code_Hospital  \
0 -1.735116      -1.244013           -0.524763   
1 -1.734423      -1.932504            0.093487   
2 -1.733729      -1.014516           -1.143013   
3 -1.733036       0.821461            0.000174   
4 -1.732343       0.821461           -0.833888   

   Available Extra Rooms in Hospital  patientid  City_Code_Patient  \
0                           0.222222  -0.965397           0.029048   
1                           0.111111  -0.965397           0.029048   
2                           0.111111  -0.965397           0.029048   
3                           0.111111  -0.965397           0.029048   
4                           0.111111  -0.965397           0.029048   

   Visitors with Patient       Age  Admission_Deposit  Stay  ...  \
0              -0.736972  0.246822           0.016381     4  ...   
1              -0.736972  0.246822           1.034451     3  ...   
2              -0.736972  0.246822          -0.145651     2  ...   
3     

In [5]:
# for tempora and binary 



hi
